##### importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy.stats as stat
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
data=pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')
data.head()

In [ ]:
print(data.dtypes)

In [ ]:
print(data.shape)

In [ ]:
#### check null values
print(data.isnull().sum())

In [ ]:
cat_features=[n for n in data.columns if data[n].dtypes=='O']
num_features=[i for i in data.columns if data[i].dtypes!="O"]
print('cateogrical_features:\n{}'.format(cat_features),'\n')
print('numerical_features:\n{}'.format(num_features))

In [ ]:
#### unique values for cateogrical_features
for i in cat_features:
    print('this feature is {}, and {},labels'.format(i,len(data[i].unique())))

In [ ]:
data.describe()

In [ ]:
cont_features=[i for i in num_features if data[i].dtypes=='float64']
dis_features=[n for n in num_features if data[n].dtypes!='float64']
print('contnious_features:\n{}'.format(cont_features),'\n')
print('discrete_features:\n{}'.format(dis_features))

In [ ]:
#### discrete_features
for feature in dis_features:
    data.groupby(feature)['Selling_Price'].median().plot.bar()
    plt.title(feature)
    plt.show()

In [ ]:
#### continous_features
for n in cont_features:
    data[n].hist(bins=25)
    plt.title(n)
    plt.show()

In [ ]:
##### boxplot
#### continous_features
for n in cont_features:
    plt.boxplot(data[n])
    plt.title(n)
    plt.show()

In [ ]:
Data=data.copy()

In [ ]:
for feature in cont_features:
    Data[feature]=np.log(Data[feature])

In [ ]:
#### relationship between contious_features and Selling_price
for feature in cont_features:
    plt.scatter(Data[feature],Data['Selling_Price'],c='red',s=100)
    plt.title(feature)
    plt.show()

In [ ]:
plt.subplot(1,2,1)
plt.hist(Data['Selling_Price'],bins=25)
plt.subplot(1,2,2)
stat.probplot(Data['Selling_Price'],dist='norm',plot=plt)

In [ ]:
##### relationship between cateogrical_features and Selling_price
for feature in cat_features:
    Data.groupby(feature)['Selling_Price'].median().plot.bar()
    plt.title(feature)
    plt.show()

In [ ]:
Data.head()

In [ ]:
##### we will skip car_name
Data=Data.iloc[:,1:]
Data.head()

In [ ]:
#### one hot encoding
Data=pd.get_dummies(Data,drop_first=True)  ### to avoid dummy variable trap
Data.head()

In [ ]:
Data.shape

In [ ]:
y=Data['Selling_Price']
X=Data.drop('Selling_Price',axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

In [ ]:
#####  important features
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print('feature_importances:\n{}'.format(model.feature_importances_))

In [ ]:
top_feat=pd.Series(model.feature_importances_,index=X.columns)
top_feat.nlargest(5).plot(kind='barh')
plt.title('feature_importance')

In [ ]:
features=top_feat.nlargest(5).index
features

In [ ]:
df=pd.concat([X[features],Data['Selling_Price']],axis=1)
df.head()

In [ ]:
#### pairplot
import seaborn as sns
sns.pairplot(df)

In [ ]:
X_features=df.drop('Selling_Price',axis=1)
y_target=df['Selling_Price']
X_features.head()

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_features,y_target,random_state=21)

In [ ]:
from xgboost import XGBRegressor
xg=XGBRegressor()
xg.fit(X_train,y_train)

In [ ]:
y_pred=xg.predict(X_test)
y_pred

In [ ]:
print('Accuracy_test:{:.3f}'.format(xg.score(X_test,y_test)))

In [ ]:
plt.figure()
sns.scatterplot(y_test,y_pred)

In [ ]:
plt.figure()
sns.distplot(y_test-y_pred)